In [1]:
from kl.utils import load_fx
import torch
import numpy as np
from kl.dichotomy_vae import DichotomyVAE, LossType
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime
from sklearn.metrics import accuracy_score

In [2]:
def train_dichotomy_vae(X, y_high, y_low, y_close, _num_epochs, _latent_dim):
    input_dim = X.shape[1]
    vae_model = DichotomyVAE(input_dim=input_dim, output_dim=input_dim, latent_dim=_latent_dim, num_classes=2, loss_type = LossType.Classifier, verbose=False)
    optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=5000)
    
    X_Tensor = torch.tensor(X, dtype=torch.float)
    y_tensor_high = torch.tensor(y_high, dtype=torch.long)
    y_tensor_low = torch.tensor(y_low, dtype=torch.long)
    y_tensor_close = torch.tensor(y_close, dtype=torch.long)
    
    # print(f'X_Tensor.shape: {X_Tensor.shape}, y_tensor_high: {y_tensor_high.shape}, y_tensor_close: {y_tensor_close.shape}, y_tensor_low: {y_tensor_low.shape}')
    
    
    # Create a DataLoader for batching
    batch_size = 64
    dataset = TensorDataset(X_Tensor, y_tensor_high, y_tensor_low, y_tensor_close)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    start_time = datetime.now()
    # Fit and transform
    vae_model.fit(dataloader, optimizer, scheduler, num_epochs=_num_epochs, beta=1, lambda_class=1)
    end_time = datetime.now()
    # Execution time
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time}")
    return vae_model

def to_np(x_reconstructed, latent_representation, predicted_class_high, predicted_class_low, predicted_class_close):
    x_reconstructed = x_reconstructed.cpu().detach().numpy()
    latent_representation = latent_representation.detach().cpu().numpy()
    predicted_class_high = predicted_class_high.detach().cpu().numpy()
    predicted_class_low = predicted_class_low.detach().cpu().numpy()
    predicted_class_close = predicted_class_close.detach().cpu().numpy()
    
    
    return x_reconstructed, latent_representation, predicted_class_high, predicted_class_low, predicted_class_close



In [3]:
window_size = 10
pair = 'EURUSD'
latent_dim = 30
num_epochs = 500
# use default shift = 1
X, y_high, y_low, y_close, returns = load_fx(data_start=0, data_end=6000, shift=2, window_size=window_size, pair=pair)
# expanding window
n_correct_high, n_correct_low, n_correct_close  = 0, 0, 0
counter = 0
for idx in range(5000, 5100):#np.shape(X)[0]):
    # train window
    X_wnd_trn = X[0:idx]
    y_high_wnd_trn = y_high[0:idx]
    y_low_wnd_trn = y_low[0:idx]
    y_close_wnd_trn = y_close[0:idx]
    
    returns_wnd_trn = returns[0:idx]
    # test window
    X_wnd_tst = X[0:idx+1]
    y_high_wnd_tst = y_high[0:idx+1]
    y_low_wnd_tst = y_low[0:idx+1]
    y_close_wnd_tst = y_close[0:idx+1]
    
    returns_wnd_tst = returns[0:idx+1]
    X_Tensor = torch.tensor(X_wnd_tst, dtype=torch.float)
    # train model
    vae_model =  train_dichotomy_vae(X=X_wnd_trn,y_high=y_high_wnd_trn,y_low=y_low_wnd_trn,y_close=y_close_wnd_trn, _num_epochs=num_epochs, _latent_dim=latent_dim)
    # eval model
    reconstructed_list = []
    # predicted_class_list = []
    train_high_acc_list = []
    train_low_acc_list = []
    train_close_acc_list = []
    for sample in range(10):
        x_reconstructed, latent_representation, predicted_class_high, predicted_class_low, predicted_class_close = vae_model.transform(X_Tensor) 
        x_reconstructed, latent_representation, predicted_class_high, predicted_class_low, predicted_class_close = to_np(x_reconstructed, latent_representation, predicted_class_high, predicted_class_low, predicted_class_close)
        
        # print(y_pred_prob[-1,:])
        # Calculate Accuracy Score for High Low Close
        acc_trn_high = accuracy_score(y_high_wnd_tst, predicted_class_high)
        acc_trn_low = accuracy_score(y_low_wnd_tst, predicted_class_low)
        acc_trn_close = accuracy_score(y_close_wnd_tst, predicted_class_close)
        
        # Append Accuracy Scores to the lists
        train_high_acc_list.append(acc_trn_high)
        train_low_acc_list.append(acc_trn_low)
        train_close_acc_list.append(acc_trn_close)   
        
        # reconstructed_list.append(x_reconstructed[-1,:])
        # predicted_class_list.append(predicted_class_close[-1])
        
    # reconstructed_list_mx = np.array(reconstructed_list)
    train_high_acc_list_mx = np.array(train_high_acc_list)
    train_low_acc_list_mx = np.array(train_low_acc_list)
    train_close_acc_list_mx = np.array(train_close_acc_list)
    
    counter += 1
    # Majority Vote
    p_class_high, p_class_low, p_class_close = 0, 0, 0
    if np.mean(train_high_acc_list_mx) > 0.5:
        p_class_high = 1
    if np.mean(train_low_acc_list_mx) > 0.5:
        p_class_low = 1
    if np.mean(train_close_acc_list_mx) > 0.5:
         p_class_close = 1
        
    # accuracy test count
    if y_high_wnd_tst[-1] == p_class_high:
        n_correct_high += 1
    if y_low_wnd_tst[-1] == p_class_low:
        n_correct_low += 1    
    if y_close_wnd_tst[-1] == p_class_close:
        n_correct_close += 1
    
    
        
    print(f"Train Accuracy | High : {np.mean(train_high_acc_list_mx)} | Low : {np.mean(train_low_acc_list_mx)} | Close : {np.mean(train_close_acc_list_mx)}, ")    
    print(f"Test Accuracy for step {counter} is High : {n_correct_high/counter} | Low : {n_correct_low/counter} | Close : {n_correct_close/counter}")
    # print(predicted_class_list_mx)
    # print(reconstructed_list_mx.shape)
    # print(np.shape(X_wnd_trn), np.shape(y_wnd_trn), np.shape(returns_wnd_trn))
    # print(np.shape(X_wnd_tst), np.shape(y_wnd_tst), np.shape(returns_wnd_tst))
    # print(np.shape(x_reconstructed), np.shape(latent_representation), np.shape(predicted_class))

Execution time: 0:00:42.015002
Train Accuracy | High : 0.9065186962607479 | Low : 0.9297140571885624 | Close : 0.8992001599680064, 
Test Accuracy for step 1 is High : 1.0 | Low : 1.0 | Close : 1.0
Execution time: 0:00:38.834943
Train Accuracy | High : 0.9126749300279886 | Low : 0.9307876849260296 | Close : 0.9056377449020392, 
Test Accuracy for step 2 is High : 1.0 | Low : 1.0 | Close : 0.5
Execution time: 0:00:39.914087
Train Accuracy | High : 0.9010593643813711 | Low : 0.9234459324405359 | Close : 0.9161103337997203, 
Test Accuracy for step 3 is High : 0.6666666666666666 | Low : 0.6666666666666666 | Close : 0.6666666666666666
Execution time: 0:00:40.991952
Train Accuracy | High : 0.8995603517186253 | Low : 0.9285371702637889 | Close : 0.9134892086330935, 
Test Accuracy for step 4 is High : 0.75 | Low : 0.75 | Close : 0.75
Execution time: 0:00:41.296837
Train Accuracy | High : 0.9158241758241757 | Low : 0.9167032967032966 | Close : 0.9056343656343657, 
Test Accuracy for step 5 is High